In [41]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Quick analysis on Aid Data

### About this analysis

I downloaded the AIDData 2.1 and 3.0 from <a href="http://aiddata.org/country-level-research-datasets">here</a>. It essentially tracks International Development Finance, i.e. who gave how much aid to who and for what. Parts of the data is questionable (as is to be expected from what I assume is a consolidation from disparate sources, some even paper-based). I was primarily interested in the following:

* **The flow of money from donors to recipient countries**
* **The percent of the committed money that actually gets disbursed (this was a big issue with the aid that committed to Nepal for Earthquake reconstruction)**
* **The portion of aid received that goes into the actual project that it was meant for**
* **A neat way to visualize all of this information**

**`Why?`** : I had 2 six-hour-flights this past week, and I needed something to do.

### About the data

AidData appears to be a pretty comprehensive repository of aid finance [It is not my domain so I am not sure if there are better sources]. Version 2.1 appears to have data from 1947-2012 while 3.0 appears to have 2013 data as well. Version 3.0 appears to be a research release version (I took that to mean beta). The fields are similar in both versions. It tracks 67 variables associated with development finance projects as it pertains to 96 donors (countries and organizations like UN, etc.). The fields that I ended up using were:

* disbursed amount
* committed amount
* total project cost 
* donor 
* recipient
* sector

Tbh, I did not take the time to read through the data dictionary properly. I just skimmed through it, couldn't find exact matches to some of the fields I was looking for, but haven't spent enough time to fairly comment on how good it actually is. The dictionary for the latest release can be found <a href="http://aiddata.org/sites/default/files/inline/readme.pdf">here</a>

### A. How aid flows across countries

For the **top 7 sectors** as measured by commitment amount (in billions USD)\*, how aid flows from the **top 5 donors** to the **top 10 recipients**. I used a **Sankey diagram** to represent this information. The d3 visualization allows you to hover over the network to see the actual value, and also move the countries around to make the graph cleaner. 


*Worth noting that this is not a flow from a donor to a recipient through a sector. Rather, a representation of what kind of sectors are attracting the most amount of donor money, and how these sectors fare in the recipients' priorities.*

In [19]:
from IPython.display import IFrame
IFrame('sankey_aid_data_2016.html',width=750,height=550, scroll=False,display='block',lineheight=0)

Pictured above is the results for AidData3.0. The values are in USD constant (adjusted for inflation). Although I am not too confident on 3.0's data quality, it appears as if World Bank (IBRD and IDA) have stepped up their funding considerably in 2013 (when I compare it to a similar graph for 2.0). 

It's insightful as to how for a country like **Japan**, Energy and Transportation are top funding priorities while for the **U.S.**, it is Health, Government and Budget. 

### B. How Nepal receives Aid. 

The **top 10 sectors** for which Nepal was committed funding from it's **top 20 donors** (as measured in millions USD). Just for comparison, I threw in Sri Lanka as well (given both share comparable recent political history, population size, geo-political stresses). 

In [48]:
IFrame('sankey_aid_data_nep_2016.html',width=750,height=550)

The graph above is for AidData3.0. Perhaps the fact that SriLanka (smaller country than Nepal, and by many counts, more developed) was committed more aid has something to do with the 2004 Tsunami (just speculating wildly here). 

Japan seems to value Transporation a lot. I would have thought health, education, etc. would be the largest sectors, but turns out Transporation, Energy, Agriculture actually are. 

### C. Measuring inefficiency and corruption: The difference between disbursed amount and project cost

I used aid disbursed as a proxy for received money (as received amount was of poor quality), and total project cost as the money that went into the project. To capture the percent of the disbursed amount that ends up not being used for the project, I used:

$\hspace{40 mm}{\text{inefficiency}= \big|\,{1-\big(\frac{\text{project cost}}{\text{disbursed amount}}}\big)\,\big|}$


The major reasons I can imagine -again, my own ignorance probably- for the disbursed amount not matching up to the project cost are:
* the money got lost during the project i.e. receiver corruption 
* donor did not disburse all of the money but the project went ahead anyway i.e. donor corruption 
* project ended up costing way more than intended i.e. inefficiency

I used the **average inefficiency** metric for countries with **at least 3** *qualifying* projects to calculate a list of the 10 most inefficient receivers to come up with the following tree-map. Clicking on recipients shows the donors associated. Color gradient shows inefficiency metric between the donor and recipient with darker being higher.

In [31]:
from IPython.display import IFrame
IFrame('treemap_dis_pc.html',width=850,height=540)

The receivers are all African, bar Syria and Baharain. But the interesting part is that most of the donors involved are associated with the middle-east somehow (i.e. Kuwait, Arab Bank, OPEC). You could summarise this as - **A lot of money seems to get lost on projects where Arab countries/organizations are donating to Africa**. Two possibilities:
* The disbursed amount and project cost data is closely tracked for these countries only (so only these show up)
* Oil is involved somehow.

### C. Measuring failure to execute: The difference between committed amount and disbursed amount

Often, aid gets committed but not disbursed. Few reasons I can think of :
* The donor is using aid as a political bargaining chip and won't release it until conditions are met.
* The reciever doesn't have their sh\*t together (aka Nepal post-2015 earthquake).
* Donor is using it as a cheap popularity stunt without intentions of following through.

I used the following metric to measure this: 

$\hspace{40 mm}{\text{inexecution}= \big|\,{1-\big(\frac{\text{ disbursed amount}}{\text{committed amount}}}\big)\,\big|}$


I used the **average inexecution** metric for countries with **more than 5** *qualifying* projects to calculate a list of the 10 most untrustworthy donors (when it comes to followingthrough). Clicking on the donors in the tree-map will reveal the receivers involved. Color gradient shows inexecution metric between the donor and recipient with darker being higher.

In [40]:
IFrame('treemap_com_dis.html',width=600,height=530)

There are a few (4, to be precise) cases where the disbursed amount is higher than the commitment amount. The dark block between Australia and Croatia is one of those. Not sure why this would happen, but the disbursed amount was only 960k USD. OPEC and Arab Bank feature among the top 10 again. Honorable mentions (Bill & Melinda Gates Foundation at 11th, United States at 14th).

Other notes:

**I did a host of basic processing to clean the data (e.g. for sector Primary and secondary education were converted to "Education" etc, nulls converted to zeros, both large and small values removed, etc. - the usual).** 

**\*Obviously, we could have used received or disbursed amount but version 3.0 of the data doesn't seem to have that, and the quality of received amount is questionable in version 2.1).**